# CONFIGURATION

In [ ]:
# Cell [1] - ĐÃ CẬP NHẬT
import os

MODEL_NAMES = [
    "joeddav/xlm-roberta-large-xnli",
    "microsoft/infoxlm-large",
    "uitnlp/CafeBERT",
    "FacebookAI/xlm-roberta-large",
    "MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli",
    "MoritzLaurer/ernie-m-large-mnli-xnli",
    "microsoft/deberta-xlarge-mnli",
]


class Config:
    ROOT_DIR = os.getcwd()
    DATA_DIR = os.path.join(ROOT_DIR, "data")

    # --- THAY ĐỔI QUAN TRỌNG ---
    # Trỏ đến file đã được xử lý bằng semantic search
    TRAIN_FILE = os.path.join(DATA_DIR, "vihallu-train.csv")

    TEST_FILE = os.path.join(DATA_DIR, "vihallu-public-test.csv")
    SUBMISSION_DIR = os.path.join(ROOT_DIR, "submission")
    SUBMISSION_CSV = "submit.csv"
    SUBMISSION_ZIP = "submit.zip"

    MODEL_NAME = MODEL_NAMES[2]
    MODEL_OUTPUT_DIR = os.path.join(
        ROOT_DIR, "models", f"{MODEL_NAME.split('/')[-1]}-tuned"
    )

    MAX_LENGTH = 512
    RANDOM_STATE = 42
    EPOCHS = 10
    BATCH_SIZE = 1
    GRADIENT_ACCUMULATION_STEPS = 16
    SCHEDULER_TYPE = "cosine"
    LEARNING_RATE = 8e-6  # Giảm LR một chút cho PET
    WEIGHT_DECAY = 0.02
    CLASSIFIER_DROPOUT = 0.05
    EPSILON = 1e-8
    PATIENCE_LIMIT = 2
    TOTAL_STEP_SCALE = 0.1
    LABEL_SMOOTHING = 0.05
    VALIDATION_SPLIT_SIZE = 0.2

    # Giữ nguyên mapping để đọc dữ liệu, nhưng sẽ được ánh xạ lại trong PET
    LABEL_MAP = {"intrinsic": 0, "extrinsic": 1, "no": 2}
    ID2LABEL = {v: k for k, v in LABEL_MAP.items()}
    CLASS_WEIGHTS = [1.0393466963622866, 1.0114145354717525, 0.9531590413943355]


cfg = Config()


# LOGGER

In [ ]:
import logging
import os
from datetime import datetime

# Thư mục gốc để lưu tất cả các file log
LOG_BASE_DIR = "logs"

# Dùng một dictionary để lưu các logger đã tạo, tránh việc tạo lại và gây ra log trùng lặp
_loggers = {}


def setup_logger(model_name: str, log_level=logging.INFO):
    """
    Thiết lập và trả về một logger để ghi log vào cả console và file.

    - Mỗi model sẽ có một thư mục log riêng dựa trên `model_name`.
    - Mỗi lần chạy sẽ tạo một file log mới có tên là timestamp (ví dụ: 2023-10-27_15-30-00.log).
    - Đảm bảo không có log nào bị ghi đè.

    Args:
        model_name (str): Tên của model, dùng để tạo thư mục con. Ví dụ: 'xnli-large-tuned'.
        log_level (int): Cấp độ log, mặc định là logging.INFO.

    Returns:
        logging.Logger: Instance của logger đã được cấu hình.
    """
    # Nếu logger cho model này đã tồn tại, trả về nó ngay lập tức
    if model_name in _loggers:
        return _loggers[model_name]

    # Xử lý tên model để an toàn khi tạo tên thư mục (thay thế "/")
    safe_model_name = model_name.replace("/", "_").replace("\\", "_")
    model_log_dir = os.path.join(LOG_BASE_DIR, safe_model_name)
    os.makedirs(model_log_dir, exist_ok=True)

    # Tạo logger
    logger = logging.getLogger(safe_model_name)
    logger.setLevel(log_level)

    # Ngăn không cho log lan truyền đến root logger để tránh in ra console 2 lần
    logger.propagate = False

    # Định dạng cho log message
    formatter = logging.Formatter(
        "%(asctime)s - [%(levelname)s] - %(message)s", datefmt="%Y-%m-%d %H:%M:%S"
    )

    # Tạo File Handler để ghi log ra file
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    log_file_path = os.path.join(model_log_dir, f"{timestamp}.log")

    file_handler = logging.FileHandler(log_file_path, encoding="utf-8")
    file_handler.setLevel(log_level)
    file_handler.setFormatter(formatter)

    # Tạo Console (Stream) Handler để in log ra màn hình
    console_handler = logging.StreamHandler()
    console_handler.setLevel(log_level)
    console_handler.setFormatter(formatter)

    # Thêm các handler vào logger
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    # Lưu logger vào cache
    _loggers[model_name] = logger

    logger.info(
        f"Logger cho '{safe_model_name}' đã được khởi tạo. File log: {log_file_path}"
    )

    return logger


## Setup logger

In [3]:
logger = setup_logger(f"{cfg.MODEL_NAME}-training")
logger.info(f"Logger initialized for {cfg.MODEL_NAME}")

logger.info("=" * 60)
logger.info("🚀 STARTING TRAINING SESSION")
logger.info("=" * 60)
for key, value in Config.__dict__.items():
    if not key.startswith("__") and not callable(value):
        logger.info(f"{key}: {value}")
logger.info("=" * 60)


2025-10-16 21:44:15 - [INFO] - Logger cho 'uitnlp_CafeBERT-training' đã được khởi tạo. File log: logs/uitnlp_CafeBERT-training/2025-10-16_21-44-15.log
2025-10-16 21:44:15 - [INFO] - Logger initialized for uitnlp/CafeBERT
2025-10-16 21:44:15 - [INFO] - ============================================================
2025-10-16 21:44:15 - [INFO] - 🚀 STARTING TRAINING SESSION
2025-10-16 21:44:15 - [INFO] - ============================================================
2025-10-16 21:44:15 - [INFO] - ROOT_DIR: /home/guest/Projects/CS221
2025-10-16 21:44:15 - [INFO] - DATA_DIR: /home/guest/Projects/CS221/data
2025-10-16 21:44:15 - [INFO] - TRAIN_FILE: /home/guest/Projects/CS221/data/vihallu-train.csv
2025-10-16 21:44:15 - [INFO] - TEST_FILE: /home/guest/Projects/CS221/data/vihallu-public-test.csv
2025-10-16 21:44:15 - [INFO] - SUBMISSION_DIR: /home/guest/Projects/CS221/submission
2025-10-16 21:44:15 - [INFO] - SUBMISSION_CSV: submit.csv
2025-10-16 21:44:15 - [INFO] - SUBMISSION_ZIP: submit.zip
202

## Prepare data

In [ ]:
from sklearn.model_selection import train_test_split


def prepare_data(config, logger=None):
    df = pd.read_csv(config.TRAIN_FILE)
    print(f"✅ Đọc thành công {len(df)} mẫu từ file đã xử lý: {config.TRAIN_FILE}")

    # Tạo 2 cột premise và hypothesis từ ngữ cảnh (context)
    df["premise"] = (
        "Câu hỏi: "
        + df["prompt"].astype(str)
        + " Ngữ cảnh: "
        + df["context"].astype(str)
    )
    df["hypothesis"] = df["response"].astype(str)

    df["label_id"] = df["label"].map(config.LABEL_MAP)
    df.dropna(subset=["label_id"], inplace=True)
    df["label_id"] = df["label_id"].astype(int)

    train_df, val_df = train_test_split(
        df,
        test_size=config.VALIDATION_SPLIT_SIZE,
        random_state=config.RANDOM_STATE,
        stratify=df["label_id"],
    )

    if logger:
        logger.info(
            f"Chia dữ liệu: {len(train_df)} mẫu train, {len(val_df)} mẫu validation."
        )

    # --- PHẦN NÂNG CẤP: LƯU FILE RA THƯ MỤC DATA ---
    # Tạo thư mục 'processed' trong 'data' nếu chưa có
    processed_data_dir = os.path.join(config.DATA_DIR, "processed")
    os.makedirs(processed_data_dir, exist_ok=True)

    # Định nghĩa đường dẫn file
    train_output_path = os.path.join(processed_data_dir, "train_split.csv")
    val_output_path = os.path.join(processed_data_dir, "validation_split.csv")

    # Lưu các DataFrame
    train_df.to_csv(train_output_path, index=False, encoding="utf-8-sig")
    val_df.to_csv(val_output_path, index=False, encoding="utf-8-sig")

    print(f"✅ Đã lưu tập train vào: {train_output_path}")
    print(f"✅ Đã lưu tập validation vào: {val_output_path}")
    # --- KẾT THÚC PHẦN NÂNG CẤP ---

    return train_df, val_df


# Model (Cập nhật cho Masked Language Modeling)

In [5]:
from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForMaskedLM,
)  # <-- THAY ĐỔI Ở ĐÂY


def get_model_and_tokenizer(config):
    """Tải pre-trained model và tokenizer cho Masked LM."""
    print(f"Đang tải model: {config.MODEL_NAME}")
    tokenizer = AutoTokenizer.from_pretrained(config.MODEL_NAME)

    cfg_model = AutoConfig.from_pretrained(config.MODEL_NAME, trust_remote_code=True)
    print(f"Model config: {cfg_model}")

    # --- THAY ĐỔI QUAN TRỌNG ---
    # Chuyển từ SequenceClassification sang MaskedLM
    model = AutoModelForMaskedLM.from_pretrained(
        config.MODEL_NAME,
        config=cfg_model,
    )
    # -------------------------

    return model, tokenizer


# Thiết lập Pattern & Verbalizer cho PET

In [6]:
import torch


def setup_pet_components(tokenizer, logger):
    """
    Định nghĩa Pattern và Verbalizer cho phương pháp PET.
    """
    # 1. Định nghĩa Pattern (Mẫu câu)
    # --- THAY ĐỔI QUAN TRỌNG: Đưa {mask} lên đầu ---
    pattern = '{mask}! Dựa trên thông tin: "{premise}", câu trả lời "{hypothesis}" có đúng không?'
    # Bạn cũng có thể thử các pattern khác như:
    # pattern = "Phán quyết: {mask}. Bằng chứng: \"{premise}\". Giả thuyết: \"{hypothesis}\"."
    # ------------------------------------------------

    logger.info(f"Sử dụng Pattern (đã sửa lỗi): {pattern}")

    # 2. Định nghĩa Verbalizer (Giữ nguyên)
    verbalizer = {
        2: "chuẩn",  # no Thay "đúng" bằng "chuẩn" hoặc "chính xác"
        0: "trật",  # intrinsic
        1: "khác",  # extrinsic
    }
    logger.info(f"Sử dụng Verbalizer: {verbalizer}")

    # 3. Lấy token ID cho các từ trong verbalizer
    verbalizer_token_ids = {
        label_id: tokenizer.convert_tokens_to_ids(verb)
        for label_id, verb in verbalizer.items()
    }

    # Kiểm tra xem có từ nào bị tách thành nhiều sub-token không
    for label_id, verb in verbalizer.items():
        tokens = tokenizer.tokenize(verb)
        if len(tokens) > 1:
            logger.warning(f"⚠️ Từ '{verb}' bị tách thành nhiều token: {tokens}.")
        else:
            logger.info(
                f"✅ Từ '{verb}' (ID: {verbalizer_token_ids[label_id]}) là token đơn lẻ."
            )

    return pattern, verbalizer, verbalizer_token_ids


# Hallucination Dataset (Cập nhật cho PET)

In [7]:
import torch
from torch.utils.data import Dataset


class HallucinationDataset(Dataset):
    def __init__(self, premises, hypotheses, labels, tokenizer, max_len, pattern):
        self.premises = premises
        self.hypotheses = hypotheses
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.pattern = pattern

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        premise = self.premises[idx]
        hypothesis = self.hypotheses[idx]
        label = self.labels[idx]

        # 1. Tạo câu prompt hoàn chỉnh bằng cách điền premise và hypothesis vào pattern
        prompt_text = self.pattern.format(
            premise=premise, hypothesis=hypothesis, mask=self.tokenizer.mask_token
        )

        # 2. Tokenize câu prompt đã tạo
        encoding = self.tokenizer(
            prompt_text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding=False,
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=False,
        )

        return {
            "input_ids": encoding["input_ids"],
            "attention_mask": encoding["attention_mask"],
            "labels": label,  # Trả về số nguyên để DataCollator xử lý
        }


# Training

In [8]:
import os
import math
import torch
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from tqdm.auto import tqdm
from torch.optim import AdamW
from huggingface_hub import login
from transformers import get_scheduler
from torch.utils.data import DataLoader
from tqdm.contrib.logging import logging_redirect_tqdm
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, accuracy_score, classification_report
from functools import partial


## Train & Evaluate Functions (Cập nhật cho PET)

In [9]:
def train_one_epoch(
    model,
    data_loader,
    loss_fn,
    optimizer,
    scheduler,
    device,
    tokenizer,
    verbalizer_ids_tensor,
    label_map,
    epoch=None,
    total_epochs=None,
    gradient_accumulation_steps=1,
):
    model.train()
    total_loss = 0
    desc = f"Train" if epoch is None else f"Epoch {epoch}/{total_epochs}"
    progress_bar = tqdm(
        data_loader, desc=desc, leave=False, dynamic_ncols=True, mininterval=0.5
    )

    optimizer.zero_grad()
    steps_in_epoch = len(data_loader)
    with logging_redirect_tqdm():
        for step, batch in enumerate(progress_bar):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            original_labels = batch["labels"]
            target_labels = torch.tensor(
                [label_map[l.item()] for l in original_labels], dtype=torch.long
            ).to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            mask_token_indices = (input_ids == tokenizer.mask_token_id).nonzero(
                as_tuple=True
            )[1]
            mask_logits = logits[torch.arange(logits.size(0)), mask_token_indices]
            verbalizer_logits = mask_logits[:, verbalizer_ids_tensor]

            loss = loss_fn(verbalizer_logits, target_labels)
            total_loss += loss.item()
            scaled_loss = loss / gradient_accumulation_steps
            scaled_loss.backward()

            if (step + 1) % gradient_accumulation_steps == 0 or (
                step + 1
            ) == steps_in_epoch:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()

            progress_bar.set_postfix({"loss": f"{loss.item():.4f}"})

    return total_loss / len(data_loader)


## Evaluate

In [10]:
def evaluate(
    model,
    data_loader,
    loss_fn,
    device,
    tokenizer,
    verbalizer_ids_tensor,
    label_map,
    id2label_map,
):
    model.eval()
    all_preds = []
    all_labels = []
    total_val_loss = 0
    progress_bar = tqdm(data_loader, desc="Evaluating", leave=False, dynamic_ncols=True)

    with torch.no_grad(), logging_redirect_tqdm():
        for batch in progress_bar:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            original_labels = batch["labels"]
            target_labels = torch.tensor(
                [label_map[l.item()] for l in original_labels], dtype=torch.long
            ).to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            mask_token_indices = (input_ids == tokenizer.mask_token_id).nonzero(
                as_tuple=True
            )[1]
            mask_logits = logits[torch.arange(logits.size(0)), mask_token_indices]
            verbalizer_logits = mask_logits[:, verbalizer_ids_tensor]

            loss = loss_fn(verbalizer_logits, target_labels)
            total_val_loss += loss.item()

            preds_indices = torch.argmax(verbalizer_logits, dim=-1)
            original_preds = [id2label_map[p.item()] for p in preds_indices]

            all_preds.extend(original_preds)
            all_labels.extend(original_labels.cpu().numpy())

    avg_val_loss = total_val_loss / len(data_loader)
    return all_labels, all_preds, avg_val_loss


# Main

In [11]:
# Tải biến môi trường từ file envs/.env.
dotenv_path = os.path.join(os.getcwd(), "envs", ".env")
load_dotenv(dotenv_path)
print(f"dotenv_path: {dotenv_path}")


dotenv_path: /home/guest/Projects/CS221/envs/.env


In [12]:
# lấy HF token để login
hf_token = os.getenv("HUGGING_FACE_TOKEN")

if hf_token:
    print("INFO: Tìm thấy HUGGING_FACE_TOKEN. Đang đăng nhập...")
    login(token=hf_token)
    print("INFO: Đăng nhập Hugging Face thành công.")
else:
    print(
        "WARNING: Không tìm thấy HUGGING_FACE_TOKEN trong file .env. Một số model có thể yêu cầu đăng nhập."
    )


INFO: Tìm thấy HUGGING_FACE_TOKEN. Đang đăng nhập...
INFO: Đăng nhập Hugging Face thành công.


# ==============================================================================
# 1. Chuẩn bị dữ liệu
# ==============================================================================

In [13]:
logger.info("Bắt đầu pipeline huấn luyện PET.")
logger.info("Bước 1: Chuẩn bị dữ liệu...")
train_df, val_df = prepare_data(cfg, logger=logger)


2025-10-16 21:44:18 - [INFO] - Bắt đầu pipeline huấn luyện PET.
2025-10-16 21:44:18 - [INFO] - Bước 1: Chuẩn bị dữ liệu...
2025-10-16 21:44:18 - [INFO] - Chia dữ liệu: 5600 mẫu train, 1400 mẫu validation.


✅ Đọc thành công 7000 mẫu từ file đã xử lý: /home/guest/Projects/CS221/data/vihallu-train.csv
✅ Đã lưu tập train vào: /home/guest/Projects/CS221/data/processed/train_split.csv
✅ Đã lưu tập validation vào: /home/guest/Projects/CS221/data/processed/validation_split.csv


# ==============================================================================
# 2. Tải model, tokenizer và thiết lập PET
# ==============================================================================

In [14]:
logger.info(f"Bước 2: Tải model '{cfg.MODEL_NAME}' và tokenizer...")
model, tokenizer = get_model_and_tokenizer(cfg)

logger.info(f"Thiết lập các thành phần cho PET...")
pattern, verbalizer, verbalizer_token_ids = setup_pet_components(tokenizer, logger)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

verbalizer_ids_tensor = torch.tensor(
    [verbalizer_token_ids[i] for i in sorted(verbalizer_token_ids.keys())]
).to(device)

original_label_to_verbalizer_idx = {
    label: i for i, label in enumerate(sorted(verbalizer_token_ids.keys()))
}
verbalizer_idx_to_original_label = {
    i: label for i, label in enumerate(sorted(verbalizer_token_ids.keys()))
}


2025-10-16 21:44:18 - [INFO] - Bước 2: Tải model 'uitnlp/CafeBERT' và tokenizer...


Đang tải model: uitnlp/CafeBERT


2025-10-16 21:44:20 - [INFO] - Thiết lập các thành phần cho PET...
2025-10-16 21:44:20 - [INFO] - Sử dụng Pattern (đã sửa lỗi): {mask}! Dựa trên thông tin: "{premise}", câu trả lời "{hypothesis}" có đúng không?
2025-10-16 21:44:20 - [INFO] - Sử dụng Verbalizer: {2: 'chuẩn', 0: 'trật', 1: 'khác'}
2025-10-16 21:44:20 - [INFO] - ✅ Từ 'chuẩn' (ID: 3) là token đơn lẻ.
2025-10-16 21:44:20 - [INFO] - ✅ Từ 'trật' (ID: 3) là token đơn lẻ.
2025-10-16 21:44:20 - [INFO] - ✅ Từ 'khác' (ID: 3) là token đơn lẻ.


Model config: XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "dtype": "float32",
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.57.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}



# ==============================================================================
# 3. Tạo Dataset và DataLoader
# ==============================================================================

In [15]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

logger.info("Bước 3: Tạo Dataset và DataLoader...")
train_dataset = HallucinationDataset(
    premises=train_df["premise"].to_list(),
    hypotheses=train_df["hypothesis"].to_list(),
    labels=train_df["label_id"].to_list(),
    tokenizer=tokenizer,
    max_len=cfg.MAX_LENGTH,
    pattern=pattern,  # <-- Truyền pattern vào
)
val_dataset = HallucinationDataset(
    premises=val_df["premise"].to_list(),
    hypotheses=val_df["hypothesis"].to_list(),
    labels=val_df["label_id"].to_list(),
    tokenizer=tokenizer,
    max_len=cfg.MAX_LENGTH,
    pattern=pattern,  # <-- Truyền pattern vào
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_loader = DataLoader(
    train_dataset, batch_size=cfg.BATCH_SIZE, shuffle=True, collate_fn=data_collator
)
val_loader = DataLoader(
    val_dataset, batch_size=cfg.BATCH_SIZE, collate_fn=data_collator
)
logger.info("✅ Tạo DataLoader thành công với DataCollatorWithPadding chuẩn!")


2025-10-16 21:44:21 - [INFO] - Bước 3: Tạo Dataset và DataLoader...
2025-10-16 21:44:21 - [INFO] - ✅ Tạo DataLoader thành công với DataCollatorWithPadding chuẩn!
2025-10-16 21:44:21 - [INFO] - ✅ Tạo DataLoader thành công với DataCollatorWithPadding chuẩn!


### Check dataset

In [16]:
print("\n--- Kiểm tra 1 batch dữ liệu đầu vào ---")
sample_batch = next(iter(train_loader))

print("Kích thước input_ids:", sample_batch["input_ids"].shape)
print("Kích thước attention_mask:", sample_batch["attention_mask"].shape)
print("Nhãn trong batch:", sample_batch["labels"])

# Giải mã một mẫu để xem nó trông như thế nào
decoded_text = tokenizer.decode(sample_batch["input_ids"][0], skip_special_tokens=False)
print("\nMột mẫu đã được token hóa và giải mã lại:")
print(decoded_text)
print("------------------------------------------\n")
# --- KẾT THÚC BƯỚC KIỂM TRA ---



--- Kiểm tra 1 batch dữ liệu đầu vào ---
Kích thước input_ids: torch.Size([1, 269])
Kích thước attention_mask: torch.Size([1, 269])
Nhãn trong batch: tensor([2])

Một mẫu đã được token hóa và giải mã lại:
<s><mask> ! Dựa trên thông tin: "Câu hỏi: Năm 2012, tỉ trọng đầu tư của Mỹ vào R&D là 33%, đúng không, và điều này cho thấy sự gia tăng vượt bậc so với các năm trước đó? Ngữ cảnh: Dù giá trị đầu tư vào R&D tại Mỹ cao, nó vẫn chưa đáp ứng được mục tiêu mà tổng thống Obama đề ra là 3% GDP vào thời điểm cuối nhiệm kỳ năm 2016. Sự độc tôn của Hoa Kỳ trong lĩnh vực này đang bị suy giảm, thậm chí với các quốc gia khác, như Trung Quốc, đang đẩy các hoạt động tài trợ R&D của học lên mức độ mới. Từ năm 2009 đến 2012, tỷ trọng tổng đầu tư của Hoa Kỳ vào R&D so với thế giới giảm nhẹ từ 30,5% còn 28,1%. Một vài quốc gia dành tới hơn 4% GDP cho hoạt động nghiên cứu và phát triển (Israel, Nhật Bản và Nam Hàn) và các nước khác có kế hoạch tăng GERD/GDP tới 4% năm 2020 (Phần Lan và Thuỵ Điển).", câu

In [17]:
print("\n--- Kiểm tra chi tiết 5 mẫu đầu tiên để so sánh trước và sau khi xử lý ---")

# Lấy 5 mẫu đầu tiên từ DataFrame gốc để so sánh
num_samples_to_check = 5
for i in range(num_samples_to_check):
    print(f"\n=============== MẪU {i} ===============")

    # 1. Lấy dữ liệu gốc từ DataFrame
    original_premise = train_df["premise"].iloc[i]
    original_hypothesis = train_df["hypothesis"].iloc[i]
    # Nối 2 chuỗi lại giống cách tokenizer sẽ thấy chúng
    original_combined_text = original_premise + " [SEP] " + original_hypothesis

    # 2. Lấy dữ liệu đã được xử lý từ Dataset
    processed_sample = train_dataset[i]
    processed_input_ids = processed_sample["input_ids"]

    # 3. Giải mã (decode) các input_ids đã xử lý trở lại thành văn bản
    decoded_text = tokenizer.decode(processed_input_ids, skip_special_tokens=False)

    # 4. So sánh và in kết quả
    original_token_count = len(tokenizer.encode(original_premise, original_hypothesis))
    processed_token_count = len(processed_input_ids)

    print(f"Số token gốc (ước tính): {original_token_count}")
    print(
        f"Số token sau khi xử lý (giới hạn bởi max_len={cfg.MAX_LENGTH}): {processed_token_count}"
    )

    if original_token_count > cfg.MAX_LENGTH:
        print("⚠️  CẢNH BÁO: Mẫu này đã bị cắt bớt (truncated)!")
    else:
        print("✅  OK: Độ dài mẫu nằm trong giới hạn, không bị cắt.")

    print("\n--- Văn bản GỐC  ---")
    print(original_combined_text)

    print("\n--- Văn bản SAU KHI DECODE từ input_ids ---")
    print(decoded_text)

print("\n===========================================")
print(
    "Kiểm tra hoàn tất. Hãy so sánh văn bản trên để xem có sự khác biệt ở cuối chuỗi không."
)



--- Kiểm tra chi tiết 5 mẫu đầu tiên để so sánh trước và sau khi xử lý ---

=============== MẪU 0 ===============
Số token gốc (ước tính): 216
Số token sau khi xử lý (giới hạn bởi max_len=512): 235
✅  OK: Độ dài mẫu nằm trong giới hạn, không bị cắt.

--- Văn bản GỐC  ---
Câu hỏi: Nước chấm được pha trộn bởi những nguyên liệu gì? Ngữ cảnh: Ẩm thực Việt Nam đặc trưng với việc sử dụng rất nhiều loại mắm, nước chấm từ loãng đến đặc. Mắm, nước chấm có thể dùng nguyên chất, có thể chưng lên hoặc pha chế, phối trộn với ớt, gừng hoặc tỏi, hạt tiêu, đường, chanh hoặc giấm. Người sành nội trợ thường có kinh nghiệm đặc biệt để pha chế nước chấm tùy theo món ăn. Thậm chí, cùng nguyên liệu là nước mắm, dấm, đường, tỏi, ớt, dùng để ăn với món gì thì tỷ lệ các thành phần pha chế cũng khác nhau, như khi dùng chấm rau sống thì pha nhạt, ăn với bún chả thì thêm chua. [SEP] Nước chấm còn có thể pha thêm với dầu mè để tạo hương thơm đặc biệt, hoặc sử dụng nước dừa tươi để làm dịu vị, tạo ra hương vị độc 

# ==============================================================================
# 4. Thiết lập Huấn luyện
# ==============================================================================

In [18]:
logger.info("Bước 4: Thiết lập optimizer, scheduler và loss function...")
optimizer = AdamW(
    model.parameters(),
    lr=cfg.LEARNING_RATE,
    weight_decay=cfg.WEIGHT_DECAY,
    eps=cfg.EPSILON,
)

gradient_accumulation_steps = max(1, cfg.GRADIENT_ACCUMULATION_STEPS)
num_update_steps_per_epoch = math.ceil(len(train_loader) / gradient_accumulation_steps)
num_training_steps = num_update_steps_per_epoch * cfg.EPOCHS
logger.info(
    f"Scheduler sẽ chạy trong {num_training_steps} bước ({num_update_steps_per_epoch} bước/epoch)"
)

warmup_steps = max(1, int(cfg.TOTAL_STEP_SCALE * num_training_steps))
scheduler = get_scheduler(
    cfg.SCHEDULER_TYPE,
    optimizer=optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=num_training_steps,
)
logger.info(f"Warmup steps: {warmup_steps}")

class_weights_tensor = torch.tensor(cfg.CLASS_WEIGHTS, dtype=torch.float).to(device)
loss_fn = torch.nn.CrossEntropyLoss(
    weight=class_weights_tensor, label_smoothing=cfg.LABEL_SMOOTHING
).to(device)
logger.info("Sử dụng Class Weights & Label smoothing cho hàm loss.")


2025-10-16 21:44:21 - [INFO] - Bước 4: Thiết lập optimizer, scheduler và loss function...
2025-10-16 21:44:21 - [INFO] - Scheduler sẽ chạy trong 3500 bước (350 bước/epoch)
2025-10-16 21:44:21 - [INFO] - Warmup steps: 350
2025-10-16 21:44:21 - [INFO] - Sử dụng Class Weights & Label smoothing cho hàm loss.
2025-10-16 21:44:21 - [INFO] - Scheduler sẽ chạy trong 3500 bước (350 bước/epoch)
2025-10-16 21:44:21 - [INFO] - Warmup steps: 350
2025-10-16 21:44:21 - [INFO] - Sử dụng Class Weights & Label smoothing cho hàm loss.


# ==============================================================================
# 5. Vòng lặp Huấn luyện
# ==============================================================================

In [19]:
logger.info("Bước 5: Bắt đầu vòng lặp huấn luyện...")
best_macro_f1 = 0.0
patience_counter = 0

for epoch in range(cfg.EPOCHS):
    logger.info(f"--- Epoch {epoch + 1}/{cfg.EPOCHS} ---")

    avg_train_loss = train_one_epoch(
        model,
        train_loader,
        loss_fn,
        optimizer,
        scheduler,
        device,
        tokenizer=tokenizer,
        verbalizer_ids_tensor=verbalizer_ids_tensor,
        label_map=original_label_to_verbalizer_idx,
        epoch=epoch + 1,
        total_epochs=cfg.EPOCHS,
        gradient_accumulation_steps=gradient_accumulation_steps,
    )
    logger.info(f"Loss trung bình trên tập train: {avg_train_loss:.4f}")
    logger.info(f"Current Learning Rate: {optimizer.param_groups[0]['lr']:.2e}")

    logger.info("Bắt đầu đánh giá trên tập validation...")
    val_labels, val_preds, avg_val_loss = evaluate(
        model,
        val_loader,
        loss_fn,
        device,
        tokenizer=tokenizer,
        verbalizer_ids_tensor=verbalizer_ids_tensor,
        label_map=original_label_to_verbalizer_idx,
        id2label_map=verbalizer_idx_to_original_label,
    )

    accuracy = accuracy_score(val_labels, val_preds)
    macro_f1 = f1_score(val_labels, val_preds, average="macro")

    logger.info(f"Validation Loss: {avg_val_loss:.4f}")
    logger.info(f"Validation Accuracy: {accuracy:.4f}")
    logger.info(f"Validation Macro-F1: {macro_f1:.4f}")

    report = classification_report(
        val_labels,
        val_preds,
        target_names=[cfg.ID2LABEL[i] for i in range(len(cfg.LABEL_MAP))],
        digits=4,
    )
    logger.info(f"Classification Report trên tập validation:\n{report}")

    if macro_f1 > best_macro_f1:
        best_macro_f1 = macro_f1
        patience_counter = 0
        logger.info(
            f"🎉 Macro-F1 cải thiện. Đang lưu model tốt nhất vào '{cfg.MODEL_OUTPUT_DIR}'..."
        )
        os.makedirs(cfg.MODEL_OUTPUT_DIR, exist_ok=True)
        model.save_pretrained(cfg.MODEL_OUTPUT_DIR)
        tokenizer.save_pretrained(cfg.MODEL_OUTPUT_DIR)
        logger.info("Lưu model thành công.")
    else:
        patience_counter += 1
        logger.warning(
            f"Macro-F1 không cải thiện. Patience: {patience_counter}/{cfg.PATIENCE_LIMIT}"
        )
        if patience_counter >= cfg.PATIENCE_LIMIT:
            logger.info("Early stopping! Dừng huấn luyện.")
            break

logger.info("🏁 Quá trình huấn luyện hoàn tất.")
logger.info(
    f"Model tốt nhất với Macro-F1 = {best_macro_f1:.4f} đã được lưu tại '{cfg.MODEL_OUTPUT_DIR}'"
)


2025-10-16 21:44:21 - [INFO] - Bước 5: Bắt đầu vòng lặp huấn luyện...
2025-10-16 21:44:21 - [INFO] - --- Epoch 1/10 ---


Epoch 1/10:   0%|          | 0/5600 [00:00<?, ?it/s]

2025-10-16 21:52:11 - [INFO] - Loss trung bình trên tập train: 1.0986
2025-10-16 21:52:11 - [INFO] - Current Learning Rate: 8.00e-06
2025-10-16 21:52:11 - [INFO] - Bắt đầu đánh giá trên tập validation...


Evaluating:   0%|          | 0/1400 [00:00<?, ?it/s]

2025-10-16 21:52:49 - [INFO] - Validation Loss: 1.0986
2025-10-16 21:52:49 - [INFO] - Validation Accuracy: 0.3500
2025-10-16 21:52:49 - [INFO] - Validation Macro-F1: 0.1728
/home/guest/.conda/envs/cs221/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/guest/.conda/envs/cs221/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/guest/.conda/envs/cs221/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-define

...

2025-10-16 22:01:26 - [INFO] - Validation Loss: 1.0986
2025-10-16 22:01:26 - [INFO] - Validation Accuracy: 0.3500
2025-10-16 22:01:26 - [INFO] - Validation Macro-F1: 0.1728
/home/guest/.conda/envs/cs221/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/guest/.conda/envs/cs221/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/guest/.conda/envs/cs221/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-define

Epoch 3/10:   0%|          | 0/5600 [00:00<?, ?it/s]

2025-10-16 22:09:33 - [INFO] - Loss trung bình trên tập train: 1.0986
2025-10-16 22:09:33 - [INFO] - Current Learning Rate: 7.06e-06
2025-10-16 22:09:33 - [INFO] - Bắt đầu đánh giá trên tập validation...


Evaluating:   0%|          | 0/1400 [00:00<?, ?it/s]

2025-10-16 22:10:11 - [INFO] - Validation Loss: 1.0986
2025-10-16 22:10:11 - [INFO] - Validation Accuracy: 0.3500
2025-10-16 22:10:11 - [INFO] - Validation Macro-F1: 0.1728
/home/guest/.conda/envs/cs221/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/guest/.conda/envs/cs221/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/guest/.conda/envs/cs221/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-define

# Phân phối kết quả đúng/sai theo từng lớp

In [20]:
val_label_names = [cfg.ID2LABEL[label_id] for label_id in val_labels]
pred_label_names = [cfg.ID2LABEL[pred_id] for pred_id in val_preds]
evaluation_df = pd.DataFrame(
    {
        "true_label": val_label_names,
        "predicted_label": pred_label_names,
    }
)
evaluation_df["status"] = evaluation_df.apply(
    lambda row: (
        "correct" if row["true_label"] == row["predicted_label"] else "incorrect"
    ),
    axis=1,
)
distribution_table = (
    evaluation_df.groupby(["true_label", "status"])
    .size()
    .unstack(fill_value=0)
    .rename_axis(None, axis=1)
    .reset_index()
    .sort_values("true_label")
)

# 1. Thêm cột 'total' bằng cách cộng cột 'correct' và 'incorrect'
distribution_table["total"] = (
    distribution_table["correct"] + distribution_table["incorrect"]
)

# 2. Thêm cột tỉ lệ đúng (correct_rate)
distribution_table["correct_rate"] = (
    distribution_table["correct"] / distribution_table["total"]
)

# 3. Thêm cột tỉ lệ sai (incorrect_rate)
distribution_table["incorrect_rate"] = (
    distribution_table["incorrect"] / distribution_table["total"]
)

# (Tùy chọn) Format các cột tỉ lệ thành dạng phần trăm cho dễ đọc
distribution_table["correct_rate"] = distribution_table["correct_rate"].map(
    "{:.2%}".format
)
distribution_table["incorrect_rate"] = distribution_table["incorrect_rate"].map(
    "{:.2%}".format
)

# In ra bảng kết quả
logger.info(f"Phân phối kết quả trên từng lớp:\n{distribution_table.to_string()}")

# Trong notebook, dùng display() sẽ cho bảng đẹp hơn
print("Bảng phân phối kết quả trên từng lớp:")
display(distribution_table)


2025-10-16 22:10:11 - [INFO] - Phân phối kết quả trên từng lớp:
  true_label  correct  incorrect  total correct_rate incorrect_rate
0  extrinsic        0        461    461        0.00%        100.00%
1  intrinsic      490          0    490      100.00%          0.00%
2         no        0        449    449        0.00%        100.00%


Bảng phân phối kết quả trên từng lớp:


,true_label,correct,incorrect,total,correct_rate,incorrect_rate
0,extrinsic,0,461,461,0.00%,100.00%
1,intrinsic,490,0,490,100.00%,0.00%
2,no,0,449,449,0.00%,100.00%
